# 📡 LAB 3 Phase 2: ETI Stream Processing

## 🎯 วัตถุประสงค์
- เข้าใจ ETI (Ensemble Transport Interface) format
- จำลองการแปลง I/Q data เป็น ETI stream
- วิเคราะห์โครงสร้าง ETI frames
- ติดตามสถานะ sync และ error rate

## 📋 ข้อมูลพื้นฐาน ETI
- **ETI Frame Size**: 6144 bytes
- **Frame Rate**: ~83.33 Hz (12 ms per frame)
- **Components**: Header + FIC + MSC + EOF + TIST

## ⚠️ หมายเหตุ
บน Google Colab ไม่สามารถรัน `eti-cmdline` ได้โดยตรง  
Lab นี้จะเน้น**การทำความเข้าใจ ETI structure**และ**วิเคราะห์ ETI frames**

บน Raspberry Pi จะใช้:
```bash
rtl_tcp -a localhost -p 1234 | eti-cmdline -F 185.36e6 -O dab.eti
```

## ⏱️ ระยะเวลา: 45-60 นาที

---
## 📦 ส่วนที่ 1: ติดตั้ง Dependencies

In [ ]:
!pip install numpy matplotlib bitstring -q

print("✅ Dependencies installed")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bitstring import BitArray
import struct
from datetime import datetime
import io

print("📚 Libraries imported")

---
## 📖 ส่วนที่ 2: ETI Frame Structure

### ETI Frame Layout (6144 bytes total):

```
+------------------+-------+------------------------------------------+
| Component        | Size  | Description                              |
+------------------+-------+------------------------------------------+
| ERR              | 4 B   | Error information                        |
| FSYNC            | 3 B   | Frame synchronization (0x073AB6)        |
| LIDATA           | 1 B   | Length indicator                         |
| FC               | 4 B   | Frame Characterization                   |
| NST              | 1 B   | Number of streams                        |
| FIC (Fast Info)  | 96 B  | Service/ensemble information            |
| MSC (Main Svc)   | ~6000B| Audio/data streams                       |
| EOF              | 4 B   | End of frame                             |
| TIST             | 4 B   | Time stamp                               |
+------------------+-------+------------------------------------------+
```

---
## 🔨 ส่วนที่ 3: สร้าง ETI Frame Parser Class

In [ ]:
class ETIFrameParser:
    """
    Parser สำหรับวิเคราะห์ ETI frames
    
    ETI (Ensemble Transport Interface) คือ format มาตรฐาน
    สำหรับส่งข้อมูล DAB+ ensemble
    """
    
    FRAME_SIZE = 6144  # bytes
    FSYNC_PATTERN = 0x073AB6  # Frame sync pattern
    
    def __init__(self):
        self.frame_count = 0
        self.sync_errors = 0
        self.ensemble_info = {}
    
    def parse_header(self, frame_bytes):
        """
        Parse ETI frame header
        
        TODO: แยกข้อมูล header fields:
        - ERR (4 bytes)
        - FSYNC (3 bytes) - ตรวจสอบว่าตรงกับ pattern
        - LIDATA (1 byte)
        - FC (4 bytes) - Frame Characterization
        """
        if len(frame_bytes) < 12:
            return None
        
        header = {}
        
        try:
            # TODO: อ่าน ERR (4 bytes)
            header['err'] = struct.unpack('>I', frame_bytes[0:4])[0]
            
            # TODO: อ่าน FSYNC (3 bytes)
            fsync = (frame_bytes[4] << 16) | (frame_bytes[5] << 8) | frame_bytes[6]
            header['fsync'] = fsync
            header['fsync_valid'] = (fsync == self.FSYNC_PATTERN)
            
            if not header['fsync_valid']:
                self.sync_errors += 1
            
            # TODO: อ่าน LIDATA (1 byte)
            header['lidata'] = frame_bytes[7]
            
            # TODO: อ่าน FC - Frame Characterization (4 bytes)
            fc_bytes = frame_bytes[8:12]
            header['fc'] = self._parse_fc(fc_bytes)
            
            return header
            
        except Exception as e:
            print(f"❌ Header parse error: {e}")
            return None
    
    def _parse_fc(self, fc_bytes):
        """
        Parse Frame Characterization
        
        TODO: แยกข้อมูล:
        - FCT: Frame Count
        - NST: Number of streams
        - FP: Frame Phase
        - MID: Mode Identity
        """
        fc_bits = BitArray(bytes=fc_bytes)
        
        fc = {
            'fct': fc_bits[0:8].uint,    # Frame count (8 bits)
            'ficf': fc_bits[8],           # FIC flag (1 bit)
            'nst': fc_bits[9:16].uint,    # Number of streams (7 bits)
            'fp': fc_bits[16:19].uint,    # Frame phase (3 bits)
            'mid': fc_bits[19:21].uint,   # Mode (2 bits)
            'fl': fc_bits[21:32].uint     # Frame length (11 bits)
        }
        
        return fc
    
    def extract_fic(self, frame_bytes):
        """
        แยก FIC (Fast Information Channel) data
        
        TODO: 
        - FIC เริ่มที่ byte offset 12
        - ขนาด 96 bytes (3 FIBs x 32 bytes)
        - บรรจุข้อมูล ensemble และ services
        """
        FIC_OFFSET = 12
        FIC_SIZE = 96
        
        if len(frame_bytes) < FIC_OFFSET + FIC_SIZE:
            return None
        
        # TODO: แยก FIC bytes
        fic_data = frame_bytes[FIC_OFFSET:FIC_OFFSET + FIC_SIZE]
        
        # แยกเป็น 3 FIBs (Fast Information Blocks)
        fibs = []
        for i in range(3):
            fib = fic_data[i*32:(i+1)*32]
            fibs.append(fib)
        
        return {'fic_raw': fic_data, 'fibs': fibs}
    
    def analyze_frame(self, frame_bytes):
        """
        วิเคราะห์ ETI frame ทั้งหมด
        
        TODO:
        - Parse header
        - Extract FIC
        - คำนวณ error rate
        - return analysis results
        """
        if len(frame_bytes) != self.FRAME_SIZE:
            return {'error': f'Invalid frame size: {len(frame_bytes)}'}
        
        self.frame_count += 1
        
        # TODO: Parse components
        header = self.parse_header(frame_bytes)
        fic = self.extract_fic(frame_bytes)
        
        # TODO: คำนวณ statistics
        error_rate = (self.sync_errors / self.frame_count) * 100 if self.frame_count > 0 else 0
        
        analysis = {
            'frame_number': self.frame_count,
            'header': header,
            'fic': fic,
            'sync_errors': self.sync_errors,
            'error_rate': error_rate,
            'sync_status': header['fsync_valid'] if header else False
        }
        
        return analysis
    
    def get_statistics(self):
        """
        รายงานสถิติการประมวลผล
        """
        return {
            'total_frames': self.frame_count,
            'sync_errors': self.sync_errors,
            'error_rate': (self.sync_errors / self.frame_count * 100) if self.frame_count > 0 else 0,
            'success_rate': ((self.frame_count - self.sync_errors) / self.frame_count * 100) if self.frame_count > 0 else 0
        }

print("✅ ETIFrameParser class defined")

---
## 🧪 ส่วนที่ 4: สร้าง Simulated ETI Frames สำหรับทดสอบ

In [ ]:
def generate_sample_eti_frame(frame_number=0, add_error=False):
    """
    สร้าง ETI frame ตัวอย่างสำหรับทดสอบ
    
    TODO:
    - สร้าง frame ขนาด 6144 bytes
    - ใส่ header ที่ถูกต้อง
    - สามารถจำลอง error ได้
    """
    frame = bytearray(6144)
    
    # TODO: ERR field (4 bytes) - ใส่ 0 = no error
    frame[0:4] = struct.pack('>I', 0)
    
    # TODO: FSYNC (3 bytes) - 0x073AB6
    if add_error:
        # จำลอง sync error
        fsync = 0x000000
    else:
        fsync = 0x073AB6
    
    frame[4] = (fsync >> 16) & 0xFF
    frame[5] = (fsync >> 8) & 0xFF
    frame[6] = fsync & 0xFF
    
    # TODO: LIDATA (1 byte)
    frame[7] = 0x00
    
    # TODO: FC - Frame Characterization (4 bytes)
    fct = frame_number & 0xFF  # Frame count
    ficf = 1  # FIC present
    nst = 4   # 4 streams
    fp = 0    # Frame phase
    mid = 1   # Mode I
    fl = 512  # Frame length
    
    fc_bits = BitArray(length=32)
    fc_bits[0:8] = fct
    fc_bits[8] = ficf
    fc_bits[9:16] = nst
    fc_bits[16:19] = fp
    fc_bits[19:21] = mid
    fc_bits[21:32] = fl
    
    frame[8:12] = fc_bits.bytes
    
    # TODO: FIC data (96 bytes) - ใส่ข้อมูลตัวอย่าง
    # ในความเป็นจริงจะมีข้อมูล ensemble และ services
    for i in range(12, 12 + 96):
        frame[i] = (i % 256)
    
    # TODO: MSC data (main service channel) - ใส่ข้อมูลสุ่ม
    np.random.seed(frame_number)
    frame[108:6136] = np.random.randint(0, 256, 6136-108, dtype=np.uint8)
    
    # TODO: EOF (4 bytes)
    frame[6136:6140] = b'\xFF\xFF\xFF\xFF'
    
    # TODO: TIST - Timestamp (4 bytes)
    frame[6140:6144] = struct.pack('>I', frame_number * 12000)  # 12ms per frame
    
    return bytes(frame)

# ทดสอบสร้าง frame
sample_frame = generate_sample_eti_frame(0)
print(f"✅ Sample ETI frame created: {len(sample_frame)} bytes")
print(f"   FSYNC: 0x{sample_frame[4]:02X}{sample_frame[5]:02X}{sample_frame[6]:02X}")

---
## 🔍 ส่วนที่ 5: ทดสอบ ETI Frame Parser

In [ ]:
# TODO: สร้าง parser instance
parser = ETIFrameParser()

# TODO: ทดสอบกับ frame ปกติ
print("📊 Testing with normal frame:")
normal_frame = generate_sample_eti_frame(1, add_error=False)
result = parser.analyze_frame(normal_frame)

print(f"\nFrame Analysis:")
print(f"  Frame number: {result['frame_number']}")
print(f"  Sync valid: {result['sync_status']}")
print(f"  FSYNC: 0x{result['header']['fsync']:06X}")
print(f"  Frame Count (FCT): {result['header']['fc']['fct']}")
print(f"  Number of streams: {result['header']['fc']['nst']}")
print(f"  FIC present: {result['header']['fc']['ficf']}")
print(f"  Mode: {result['header']['fc']['mid']}")

# TODO: ทดสอบกับ error frame
print("\n📊 Testing with error frame:")
error_frame = generate_sample_eti_frame(2, add_error=True)
result_error = parser.analyze_frame(error_frame)

print(f"\nFrame Analysis:")
print(f"  Frame number: {result_error['frame_number']}")
print(f"  Sync valid: {result_error['sync_status']}")
print(f"  Sync errors: {result_error['sync_errors']}")
print(f"  Error rate: {result_error['error_rate']:.2f}%")

# TODO: แสดง statistics
print("\n📈 Parser Statistics:")
stats = parser.get_statistics()
for key, value in stats.items():
    print(f"  {key}: {value}")

---
## 📊 ส่วนที่ 6: จำลองการประมวลผล ETI Stream

In [ ]:
# TODO: จำลองการรับ ETI stream แบบ continuous
NUM_FRAMES = 100  # ทดสอบ 100 frames (~1.2 วินาที)
ERROR_RATE = 0.05  # จำลอง 5% error rate

print(f"📡 Simulating ETI stream processing...")
print(f"   Frames: {NUM_FRAMES}")
print(f"   Simulated error rate: {ERROR_RATE*100}%\n")

# สร้าง parser ใหม่
stream_parser = ETIFrameParser()

# เก็บผลลัพธ์
results = []
sync_status_list = []

# TODO: ประมวลผลแต่ละ frame
for i in range(NUM_FRAMES):
    # สุ่มว่าจะมี error หรือไม่
    has_error = (np.random.random() < ERROR_RATE)
    
    # สร้าง frame
    frame = generate_sample_eti_frame(i, add_error=has_error)
    
    # วิเคราะห์
    result = stream_parser.analyze_frame(frame)
    results.append(result)
    sync_status_list.append(1 if result['sync_status'] else 0)
    
    # แสดงความคืบหน้าทุก 20 frames
    if (i + 1) % 20 == 0:
        stats = stream_parser.get_statistics()
        print(f"Frame {i+1}/{NUM_FRAMES}: " +
              f"Sync OK: {stats['success_rate']:.1f}%, " +
              f"Errors: {stats['sync_errors']}")

# TODO: แสดงสถิติสุดท้าย
final_stats = stream_parser.get_statistics()
print(f"\n✅ Stream processing completed!")
print(f"\n📈 Final Statistics:")
print(f"   Total frames: {final_stats['total_frames']}")
print(f"   Sync errors: {final_stats['sync_errors']}")
print(f"   Error rate: {final_stats['error_rate']:.2f}%")
print(f"   Success rate: {final_stats['success_rate']:.2f}%")

---
## 📈 ส่วนที่ 7: Visualization - Sync Status และ Error Rate

In [ ]:
# TODO: สร้าง plots แสดงผล
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# Plot 1: Sync Status Timeline
ax1 = axes[0]
ax1.plot(sync_status_list, 'o-', markersize=3, linewidth=0.5)
ax1.fill_between(range(len(sync_status_list)), sync_status_list, alpha=0.3)
ax1.set_xlabel('Frame Number')
ax1.set_ylabel('Sync Status')
ax1.set_title('ETI Frame Synchronization Status')
ax1.set_ylim([-0.1, 1.1])
ax1.set_yticks([0, 1])
ax1.set_yticklabels(['❌ Error', '✅ OK'])
ax1.grid(True, alpha=0.3)

# Plot 2: Cumulative Error Rate
ax2 = axes[1]
cumulative_errors = np.cumsum([0 if s else 1 for s in sync_status_list])
cumulative_frames = np.arange(1, len(sync_status_list) + 1)
error_rate_timeline = (cumulative_errors / cumulative_frames) * 100

ax2.plot(error_rate_timeline, linewidth=2, color='red', label='Error Rate')
ax2.axhline(y=ERROR_RATE*100, color='blue', linestyle='--', 
            label=f'Expected: {ERROR_RATE*100}%', alpha=0.7)
ax2.set_xlabel('Frame Number')
ax2.set_ylabel('Error Rate (%)')
ax2.set_title('Cumulative Frame Error Rate')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Visualization completed")

---
## 🔬 ส่วนที่ 8: วิเคราะห์ FIC Data Structure

In [ ]:
# TODO: วิเคราะห์ FIC data จาก frame ตัวอย่าง
sample_frame = generate_sample_eti_frame(10)
fic_result = parser.extract_fic(sample_frame)

print("📊 FIC Data Analysis:")
print(f"\nFIC Size: {len(fic_result['fic_raw'])} bytes")
print(f"Number of FIBs: {len(fic_result['fibs'])}")

# แสดง FIB structure
for i, fib in enumerate(fic_result['fibs']):
    print(f"\nFIB {i}: {len(fib)} bytes")
    print(f"  First 16 bytes (hex): {fib[:16].hex()}")

# Visualize FIC data
plt.figure(figsize=(15, 4))
fic_array = np.frombuffer(fic_result['fic_raw'], dtype=np.uint8)
plt.plot(fic_array, linewidth=1)
plt.xlabel('Byte Position')
plt.ylabel('Value')
plt.title('FIC Data Content (96 bytes)')
plt.grid(True, alpha=0.3)

# เพิ่มเส้นแบ่ง FIBs
for i in range(1, 3):
    plt.axvline(x=i*32, color='red', linestyle='--', alpha=0.5, label=f'FIB {i}' if i == 1 else '')

plt.legend()
plt.show()

print("\n💡 FIC Information:")
print("   FIC บรรจุข้อมูล:")
print("   - Ensemble label (ชื่อ ensemble)")
print("   - Service list (รายชื่อสถานี)")
print("   - Subchannel organization")
print("   - Time and date information")

---
## 💾 ส่วนที่ 9: บันทึกและโหลด ETI Stream

ในการใช้งานจริง ETI stream จะถูกบันทึกเป็นไฟล์ `.eti`

In [ ]:
def save_eti_stream(frames, filename="simulated_stream.eti"):
    """
    บันทึก ETI frames เป็นไฟล์
    
    TODO:
    - เขียน frames ต่อกันเป็นไฟล์เดียว
    - แต่ละ frame ขนาด 6144 bytes
    """
    with open(filename, 'wb') as f:
        for frame in frames:
            f.write(frame)
    
    file_size = len(frames) * 6144
    print(f"💾 Saved {len(frames)} frames to {filename}")
    print(f"   File size: {file_size:,} bytes ({file_size/1024:.1f} KB)")
    print(f"   Duration: ~{len(frames) * 12 / 1000:.2f} seconds")
    
    return filename

def load_eti_stream(filename):
    """
    โหลด ETI frames จากไฟล์
    
    TODO:
    - อ่านไฟล์ทีละ 6144 bytes
    - return list of frames
    """
    frames = []
    
    with open(filename, 'rb') as f:
        while True:
            frame = f.read(6144)
            if len(frame) < 6144:
                break
            frames.append(frame)
    
    print(f"📂 Loaded {len(frames)} frames from {filename}")
    return frames

# TODO: สร้างและบันทึก sample stream
sample_frames = [generate_sample_eti_frame(i) for i in range(50)]
eti_filename = save_eti_stream(sample_frames)

# TODO: ลองโหลดกลับมา
print("\n📂 Loading ETI stream back...")
loaded_frames = load_eti_stream(eti_filename)
print(f"✅ Verification: {len(loaded_frames)} frames loaded")

---
## 🔗 ส่วนที่ 10: การใช้งานกับ Raspberry Pi

### บน Raspberry Pi คุณจะใช้:

#### 1. เริ่ม rtl_tcp server:
```bash
rtl_tcp -a 0.0.0.0 -p 1234 -f 185.36e6 -s 2.048e6 -g 30
```

#### 2. ใช้ eti-cmdline แปลงเป็น ETI:
```bash
# เชื่อมต่อ rtl_tcp และสร้าง ETI file
eti-cmdline -I rtl_tcp://localhost:1234 \
            -F 185.36e6 \
            -O dab_ensemble.eti
```

#### 3. วิเคราะห์ ETI file ด้วย Python:
```python
# บน RPI
parser = ETIFrameParser()
frames = load_eti_stream('dab_ensemble.eti')

for frame in frames:
    result = parser.analyze_frame(frame)
    if result['sync_status']:
        print(f"Frame {result['frame_number']}: OK")
```

### ติดตั้ง eti-cmdline:
```bash
git clone https://github.com/JvanKatwijk/eti-stuff
cd eti-stuff
mkdir build && cd build
cmake .. -DRTLSDR=1
make -j4
sudo make install
```

---
## 🧹 ส่วนที่ 11: Cleanup

In [ ]:
# TODO: ลบไฟล์ชั่วคราว
import os

if os.path.exists(eti_filename):
    os.remove(eti_filename)
    print(f"🧹 Removed temporary file: {eti_filename}")

print("\n✅ Lab 3 Phase 2 completed!")

---
## 📝 สรุป Lab 3 Phase 2

### ✅ สิ่งที่เรียนรู้:
1. ✅ โครงสร้าง ETI frame (6144 bytes)
2. ✅ การ parse ETI header และ FC
3. ✅ การแยก FIC data (Fast Information Channel)
4. ✅ การติดตาม sync status และ error rate
5. ✅ การจัดเก็บและโหลด ETI stream
6. ✅ การวิเคราะห์คุณภาพสัญญาณ

### 🎯 ทักษะที่ได้รับ:
- การทำงานกับ binary data และ bit manipulation
- การ parse structured data formats
- การติดตาม stream quality metrics
- File I/O operations
- Data visualization

### 🔜 ขั้นต่อไป: Lab 3 Phase 3
- Parse FIC data เพื่อหา services
- สร้าง service list database
- แยก subchannel information
- เตรียมข้อมูลสำหรับ audio decoding

---

## 📚 เอกสารอ้างอิง
- [ETSI EN 300 799 (ETI Specification)](https://www.etsi.org/deliver/etsi_en/300700_300799/300799/01.02.01_60/en_300799v010201p.pdf)
- [eti-stuff GitHub](https://github.com/JvanKatwijk/eti-stuff)
- [DAB+ Standard ETSI EN 300 401](https://www.etsi.org/deliver/etsi_en/300400_300499/300401/02.01.01_60/en_300401v020101p.pdf)